Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "08")
cd(ProjDir)

### snippet 5.1

In [3]:
d = CSV.read(rel_path("..", "data", "rugged.csv"), delim=';');
df = convert(DataFrame, d);

dcc = filter(row -> !(ismissing(row[:rgdppc_2000])), df)
dcc[:log_gdp] = log.(dcc[:rgdppc_2000])
dcc[:cont_africa] = Array{Float64}(convert(Array{Int}, dcc[:cont_africa]))
dcc[:rugged] = convert(Array{Float64}, dcc[:rugged])
first(dcc[[:rugged, :cont_africa, :log_gdp]], 5)

,rugged,cont_africa,log_gdp
,Float64,Float64,Float64
1,0.858,1.0,7.49261
2,3.427,0.0,8.21693
3,0.769,0.0,9.93326
4,0.775,0.0,9.40703
5,2.688,0.0,7.79234


Define the Stan language model

In [4]:
m_8_1_model = "
data{
    int N;
    vector[N] log_gdp;
    vector[N] cont_africa;
    vector[N] rugged;
    vector[N] rugged_cont_africa;
}
parameters{
    real a;
    real bR;
    real bA;
    real bAR;
    real sigma;
}
model{
    vector[N] mu = a + bR * rugged + bA * cont_africa + bAR * rugged_cont_africa;
    sigma ~ uniform( 0 , 10 );
    bAR ~ normal( 0 , 10 );
    bA ~ normal( 0 , 10 );
    bR ~ normal( 0 , 10 );
    a ~ normal( 0 , 100 );
    log_gdp ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_8_1_model",
monitors = ["a", "bR", "bA", "bAR", "sigma"],
model=m_8_1_model, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan will be updated.



Input data for cmdstan

In [6]:
m_8_1_data = Dict("N" => size(dcc, 1),
"log_gdp" => dcc[:log_gdp],  "rugged" => dcc[:rugged],
"cont_africa" => dcc[:cont_africa],
"rugged_cont_africa" => dcc[:rugged] .* dcc[:cont_africa] );

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_8_1_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.22 0.14  9.00  9.46   282    1
bR    -0.21 0.08 -0.33 -0.08   275    1
bA    -1.94 0.24 -2.33 -1.59   268    1
bAR    0.40 0.14  0.18  0.62   271    1
sigma  0.96 0.05  0.87  1.04   339    1
"

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.423714, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.0216162, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/08/tmp/m_8_1_model.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either seve

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.22 0.14  9.00  9.46   282    1\nbR    -0.21 0.08 -0.33 -0.08   275    1\nbA    -1.94 0.24 -2.33 -1.59   268    1\nbAR    0.40 0.14  0.18  0.62   271    1\nsigma  0.96 0.05  0.87  1.04   339    1\n"

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD       Naive SE       MCSE      ESS
    a  9.22433759 0.13826261 0.0021861238 0.0036621415 1000
   bR -0.20252029 0.07693554 0.0012164576 0.0021115164 1000
   bA -1.94709743 0.22791568 0.0036036633 0.0066790623 1000
  bAR  0.39153155 0.13220050 0.0020902734 0.0037112656 1000
sigma  0.95059016 0.05336262 0.0008437371 0.0010097003 1000

Quantiles:
          2.5%       25.0%      50.0%      75.0%        97.5%   
    a  8.95895975  9.12941750  9.222625  9.31805250  9.495101000
   bR -0.35029115 -0.25462175 -0.202754 -0.15062850 -0.051573347
   bA -2.40285000 -2.10422750 -1.943695 -1.79249750 -1.501417750
  bAR  0.13009510  0.29951525  0.391269  0.47999525  0.654214475
sigma  0.85223143  0.91440000  0.946774  0.98449750  1.063927750



End of `08/m8.1s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*